<a href="https://colab.research.google.com/github/ajkerrigan/my-itchy-dog/blob/main/Ginger_Scratching_Trends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graphing Ginger's Scratching

Ginger has sensitive skin and allergies, aww...

We rely on allergy medication to help manage the itchiness, and use a [Whistle](https://www.whistle.com/) device to help track her scratching frequency over time. That's a very useful piece of the puzzle, but it can only show 1 week or 1 month of scratching data at a time. To gauge medication effectiveness and help plan future doses, it's helpful to look further back to see longer-term trends.

Here we fetch data from the Whistle API (cheers to [go-whistle-wrapper](https://github.com/amattu2/go-whistle-wrapper) for highlighting the methods I needed), then wrap it in a [Polars](https://pola-rs.github.io/) data frame and graph it with [Altair](https://altair-viz.github.io).

This uses Altair's [experimental support](https://altair-viz.github.io/user_guide/data.html#specifying-data) for the DataFrame Interchange Protocol under the hood, so step 0 is making sure we've got current versions of altair/pyarrow/polars installed.

In [1]:
%pip install --upgrade altair pyarrow polars

# Fetch the Data

Grab 180 days of scratching data from the Whistle API, and wrap it in a Polars DataFrame.

In [2]:
import polars as pl
import requests

from google.colab import userdata

# Hi it's me
auth_token = requests.post(
    'https://app.whistle.com/api/login',
    json={
        'email': userdata.get('email'),
        'refresh_token': userdata.get('refresh_token')
    }
).json()['auth_token']

# Where my dogs at?
pets = requests.get(
    'https://app.whistle.com/api/pets',
    headers={
        'Accept': "application/vnd.whistle.com.v4+json",
        'Authorization': f'Bearer {auth_token}'
    }
).json()['pets']

# Hey that's my dog
ginger = next(p['id'] for p in pets if p['name'] == 'Ginger')

# Itchiness diary please
scratching = requests.get(
    'https://app.whistle.com/api/pets/511006/health/graphs/scratching?num_of_days=180',
    headers={
        'Accept': "application/vnd.whistle.com.v4+json",
        'Authorization': f'Bearer {auth_token}'
    }
).json()['data']

df = pl.DataFrame(scratching)

# Graph

Whistle has four categories of scratching, color coded as green/yellow/orange/red in the app. That's a useful breakdown, so use the same scheme here.

Aside from the custom color scale and some friendly titles, this is right out of the Altair [getting started](https://altair-viz.github.io/getting_started/overview.html) material.

In [3]:
import altair as alt

scale = alt.Scale(
    domain=['infrequent', 'occasional', 'elevated', 'severe'],
    range=['green', 'yellow', 'orange', 'red']
)

alt.Chart(df, title="Ginger's Scratching").mark_bar().encode(
    x=alt.X('start_date', type='temporal').title('Date'),
    y=alt.Y('score', type='quantitative').title('Scratching Time (seconds)'),
    color=alt.Color('status', type='nominal', scale=scale)
).properties(
    width='container',
    height=600,
)

alt.Chart(...)